# __Robot Aspiradora__

In [ ]:
from pickle import FALSE
from IPython.display import display
import ipywidgets as widgets
import time
import numpy as np
import random
n = 5

class Tablero:
  def __init__(self, tamano_celda=(50, 50), n_celdas=(n,n)):
    self.out = widgets.HTML()
    display(self.out)
    self.tamano_celda = tamano_celda
    self.n_celdas = n_celdas

  def dibujar(self, objetos):
    tablero = "<table border='1' >{}</table>"
    filas = ""

    for i in range(self.n_celdas[0]):
      s = ""
      for j in range(self.n_celdas[1]):
        contenido =""
        for o in objetos:
          if o.x == j and o.y == i:
            contenido = \
            "<div style='transform: rotate({angulo}deg);font-size:{tamano_emoticon}px;'>{emoticon}</div>".\
            format(angulo = o.angulo, tamano_emoticon = o.tamano_emoticon, emoticon = o.emoticon)
        s += "<td style='height:{alto}px;width:{ancho}px'>{contenido}</td>".\
          format(alto = self.tamano_celda[0], ancho = self.tamano_celda[1],
                contenido = contenido)
      filas += "<tr>{}</tr>".format(s)
    tablero = tablero.format(filas)
    self.out.value = tablero

class Agente:
  def __init__(self, x=0, y=0, angulo=0, basura=0, energia = 5, emoticon="🤖", tamano_emoticon=30):
    self.x = x
    self.y = y
    self.angulo = angulo
    self.emoticon = emoticon
    self.tamano_emoticon = tamano_emoticon
    self.energia = energia
    self.basura = basura
    self.matriz_memoria = np.zeros((n,n))
    self.pasos_sin_limpiar = 0
    self.columna_revisada = 2
    self.hojas_recogidas = 0

  def escaneo(self, objetos):
    if self.energia > 0:
      basura=False
      for index in range(0,len(objetos)-1):
        #print("Objetos x", objetos[index].x, "Objetos y", objetos[index].y, "Self x", self.x, "Self y", self.y)
        if(self.x == objetos[index].x and self.y == objetos[index].y):
          #print("Escaneo Normal: Basura Encontrada")
          basura=True
          self.pasos_sin_limpiar = 0
          objetos.pop(index)
          self.hojas_recogidas += 1
          self.matriz_memoria[self.y][self.x] = 1
          self.energia -=1
          #print("Basura Removida\n")
          break
      if(basura == False):
        #print("Basura No Encontrada\n")
        self.matriz_memoria[self.y][self.x] = 1
    else:
      print("¡No hay energía disponible!")

  def escaneoVecino(self,objetos):
    if self.energia > 0:
      basura = False
      derecha = False
      abajo = False
      izquierda = False
      arriba = False
      #print("Self x", self.x, "Self y", self.y)
      for index in range(0,len(objetos)-1):
        #print("Objetos x", objetos[index].x, "Objetos y", objetos[index].y)
        if((self.x+1) == objetos[index].x and (self.y) == objetos[index].y):
          basura = True
          derecha = True
        if((self.x) == objetos[index].x and (self.y+1) == objetos[index].y):
          basura = True
          abajo = True
        if((self.x-1) == objetos[index].x and (self.y) == objetos[index].y):
          basura = True
          izquierda = True
        if((self.x) == objetos[index].x and (self.y-1) == objetos[index].y):
          basura = True
          arriba = True
      if(basura):
        #print("Escaneo Vecino: Basura Encontrada")
        self.energia -=1
        if(derecha):
          #print("Escaneo Vecino Derecha: Basura Encontrada")
          self.matriz_memoria[self.y][self.x+1] = 0
          angulo = 270
        else:
          if(self.x<n-1):
            #print("Escaneo Vecino Derecha: Basura No Encontrada")
            self.matriz_memoria[self.y][self.x+1] = 1
        if(abajo):
          #print("Escaneo Vecino Abajo: Basura Encontrada")
          self.matriz_memoria[self.y+1][self.x] = 0
          angulo = 0
        else:
          if(self.y<n-1):
            #print("Escaneo Vecino Abajo: Basura No Encontrada")
            self.matriz_memoria[self.y+1][self.x] = 1
        if(izquierda):
          #print("Escaneo Vecino Izquierda: Basura Encontrada")
          self.matriz_memoria[self.y][self.x-1] = 0
          angulo = 90
        else:
          if(self.x>0):
            #print("Escaneo Vecino Izquierda: Basura No Encontrada")
            self.matriz_memoria[self.y][self.x-1] = 1
        if(arriba):
          #print("Escaneo Vecino Arriba: Basura Encontrada")
          self.matriz_memoria[self.y-1][self.x] = 0
          angulo = 180
        else:
          if(self.y>0):
            #print("Escaneo Vecino Arriba: Basura No Encontrada")
            self.matriz_memoria[self.y-1][self.x] = 1
        return angulo
      elif(basura == False):
        if(self.x<n-1):
            self.matriz_memoria[self.y][self.x+1] = 1
        if(self.y<n-1):
            self.matriz_memoria[self.y+1][self.x] = 1
        if(self.x>0):
            self.matriz_memoria[self.y][self.x-1] = 1
        if(self.y>0):
            self.matriz_memoria[self.y-1][self.x] = 1
        self.pasos_sin_limpiar += 1
        #print("Escaneo Vecino: Basura No Encontrada")
        self.energia -=1
        if(self.x==0 and self.y == 0):
          angulo = 270
        elif(self.x==n-1 and self.y == n-1):
          angulo = 90
        elif(self.x==0 and self.y == n-1):
          angulo = 180
        elif(self.x==n-1 and self.y == 0):
          angulo = 0
        elif(self.x == 0 and self.y > 0 and self.y < n-1):
            angulo = 180
        elif(self.x == n-1 and self.y > 0 and self.y < n-1):
            angulo = 0
        elif(self.y == 0 and self.x > 0 and self.x < n-1):
            angulo = 270
        elif(self.y == n-1 and self.x > 0 and self.x < n-1):
            angulo = 90
        else:
            angulo = self.angulo
        if(self.pasos_sin_limpiar > (3*n) and self.x == self.columna_revisada and self.y == 0):
            self.pasos_sin_limpiar = 0
            self.columna_revisada += 2
            print("Ejecutando Limpieza Profunda Por Columnas")
            if(self.columna_revisada < n):
              angulo = 0
        return angulo


  def avanzar(self):
    if self.energia > 0:
      if self.angulo == 0:
        self.abajo()
      elif self.angulo == 90:
        self.izquierda()
      elif self.angulo == 180:
        self.arriba()
      elif self.angulo == 270:
        self.derecha()
      self.energia -=1
    else:
      print("¡No hay energía disponible!")

  def abajo(self):
    if self.y < n :
      self.y += 1
  def derecha(self):
    if self.x < n :
      self.x += 1
  def arriba(self):
    if self.y > -1 :
      self.y -= 1
  def izquierda(self):
    if self.x > -1 :
      self.x -= 1
  def girar(self,angulo):
    self.angulo = angulo

  def revisionMemoria(self):
    revisionCompleta = True
    for columna in range(len(self.matriz_memoria[0])):
      for fila in range(len(self.matriz_memoria)):
        if(self.matriz_memoria[fila - (columna % 2) * (fila + fila + 1)][columna] == 0):
            revisionCompleta = False
    return revisionCompleta




def llenadoAleatorio():
  r=0.0
  for i in range(0,n):
    for j in range(0,n):
      r=random.random()
      if(r>0.5):
        basura = Agente(i, j, 0, 1, 0, emoticon="🍂", tamano_emoticon=30)
        objetos.append(basura)


def limpieza(agente, escenario):
  while(agente.energia > 0 and not agente.revisionMemoria()):
    agente.escaneo(objetos)
    time.sleep(0.3)
    #print(agente.matriz_memoria)
    agente.girar(agente.escaneoVecino(objetos))
    agente.avanzar()
    escenario.dibujar(objetos)
hojas = np.zeros(50)
#for i in range (0,50):
escenario = Tablero()
posicionX = 2
posicionY = 2
energia = 200
agente = Agente(posicionX, posicionY, 0, 0, energia,emoticon="🤖", tamano_emoticon=30)
objetos = []
llenadoAleatorio()
escenario.dibujar(objetos)
print("Inicializando Tablero...")
time.sleep(1)
objetos.append(agente)
escenario.dibujar(objetos)
print("Inicializando Robot...")
time.sleep(1)
limpieza(agente,escenario)
if(agente.energia != 0):
  print("Limpieza Finalizada")
print("Hojas Recogidas: ", agente.hojas_recogidas)
print("Energía consumida: ", energia - agente.energia)